In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import glob
import warnings
warnings.filterwarnings('ignore')


#data
path = "../data/data_daily"
all_files = glob.glob(path + "/*.csv")   

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    date = os.path.basename(filename)[:-4]
    df['date'] = date
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df = result_df.reset_index(drop=True)
df = result_df.copy()
# result_df = result_df[(result_df['date'] >= '2021-01-01') & (result_df['date'] <= '2021-12-31')]

# Construction

In [4]:
import statsmodels.api as sm
df['Turn20'] = df.groupby(['code'])['turnover_ratio'].transform(lambda x: x.rolling(20).mean())
df['Turn_Bench'] = df.groupby(['code'])['turnover_ratio'].transform(lambda x: (x.rolling(60).sum() - x.rolling(20).sum()) / 40)
df['tmp'] = df['turnover_ratio'] / df['Turn_Bench'] - 1
df['PctTurn20'] = df.groupby(['code'])['tmp'].transform(lambda x: x.rolling(20).mean())
df['STR'] = df.groupby(['code'])['turnover_ratio'].transform(lambda x: x.rolling(20).std())


# Output

In [28]:
factor_name = "STR"
output_folder = "../data/factors" + "/" + factor_name + "/unneutralized"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

selected_col = ['code', 'Turn20', 'PctTurn20', 'STR']
# date_index = pd.date_range(df['date'].min(), df['date'].max(), freq='D')
# trade_date = [d for d in date_index if d in df['date'].unique()]
trade_date = df['date'].unique()

for date in tqdm(trade_date):
    group = df.loc[df['date'] == date, selected_col]
    file_name = os.path.join(output_folder, date+'.csv')
    group.sort_values(by='code', ascending=True, inplace=True)
    group.to_csv(file_name, index=False)

100%|██████████| 949/949 [03:52<00:00,  4.09it/s]


# Neutralize

In [36]:
from Barra_Neutralization import NeutralizationProcessor
barra_path = "../data/data_barra"
factor_path = "../data/factors/STR/unneutralized"
output_folder =  "../data/factors/STR/neutralized"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [37]:
import ray
# Usage
mode = 3
barra_col = ['size']
ray.shutdown()
ray.init(num_cpus=4, num_gpus=0)

process = NeutralizationProcessor.remote(barra_path, factor_path, barra_col, output_folder, mode)
s = time.time()
barrl_files = sorted(os.listdir(barra_path))
results = []
for f in barrl_files:
    results.append(process.process_file.remote(f))
    # process.process_file.remote(f)

ray.get(results)
e = time.time()
print("Time taken: ", e-s)

ray.shutdown()


2024-12-28 10:03:56,917	INFO worker.py:1821 -- Started a local Ray instance.
(NeutralizationProcessor pid=36144) d:\OneDrive\Study\Quant\Factor\Factor\Barra_Neutralization.py:49: RuntimeWarning: divide by zero encountered in log10
(NeutralizationProcessor pid=36144)   factor_df[col] = factor_df[col] / 10**np.ceil(np.log10(factor_df[col].abs().max()))
(NeutralizationProcessor pid=36144) d:\OneDrive\Study\Quant\Factor\Factor\Barra_Neutralization.py:49: RuntimeWarning: divide by zero encountered in log10
(NeutralizationProcessor pid=36144)   factor_df[col] = factor_df[col] / 10**np.ceil(np.log10(factor_df[col].abs().max()))
(NeutralizationProcessor pid=36144) d:\OneDrive\Study\Quant\Factor\Factor\Barra_Neutralization.py:49: RuntimeWarning: divide by zero encountered in log10
(NeutralizationProcessor pid=36144)   factor_df[col] = factor_df[col] / 10**np.ceil(np.log10(factor_df[col].abs().max()))
(NeutralizationProcessor pid=36144) d:\OneDrive\Study\Quant\Factor\Factor\Barra_Neutralization.

Time taken:  40.04225158691406
